In [1]:
## set up imports of local module code
import os
nb_path = os.path.abspath("")
from sys import path
from os.path import dirname

path.insert(0,  dirname(nb_path))

nb_path


'c:\\Users\\ethan\\coding_projects\\pypreql\\experiments'

In [2]:
### Test


from preql import Executor, Dialects
from sqlalchemy import create_engine
from preql.hooks.query_debugger import DebuggingHook
from preql.constants import logger
from logging import StreamHandler
engine = create_engine(r"duckdb:///:memory:", future=True)

test = Executor(engine=engine, dialect=Dialects.DUCK_DB)



In [3]:
import pandas as pd

df = pd.read_csv('./train.csv')

In [4]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.execute_raw_sql("CREATE TABLE raw_titanic AS SELECT * FROM df")


In [6]:
from preql.core.models import Datasource, Concept, ColumnAssignment
from preql.core.enums import DataType, Purpose
env = test.environment
namespace = 'passenger'
id = Concept(name='id', namespace=namespace, datatype=DataType.INTEGER,
                        purpose=Purpose.KEY)
age = Concept(name='age', namespace=namespace, datatype=DataType.INTEGER, purpose=Purpose.PROPERTY)

name = Concept(name='name', namespace=namespace, datatype=DataType.STRING, purpose=Purpose.PROPERTY)

pclass = Concept(name='passenger_class', namespace=namespace, purpose = Purpose.PROPERTY,
                        datatype=DataType.INTEGER
                        )
survived = Concept(name='survived', namespace=namespace, purpose = Purpose.PROPERTY,
                        datatype=DataType.BOOL
                        )
fare = Concept(name='fare', namespace=namespace, purpose = Purpose.PROPERTY, datatype=DataType.FLOAT )
for x in [id, age, survived, name, pclass, fare]:
    env.add_concept(x)

test.environment.add_datasource(
    Datasource(
        identifier = 'raw_data',
        address = 'raw_titanic',
        columns= [ColumnAssignment(alias='passengerid', concept=id ),
                  ColumnAssignment(alias='age', concept=age),
                  ColumnAssignment(alias='survived', concept=survived),
                  ColumnAssignment(alias='pclass', concept=pclass),
                  ColumnAssignment(alias='name', concept=name),
                  ColumnAssignment(alias='fare', concept=fare)]

    ), 

)

Datasource(identifier='raw_data', columns=[ColumnAssignment(alias='passengerid', concept=Concept(name='id', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=None, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='passenger', keys=None, grain=Grain(nested=False, components=[Concept(name='id', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=None, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='passenger', keys=None, grain=Grain(nested=True, components=[Concept(name='id', datatype=<DataType.INTEGER: 'int'>, purpose=<Purpose.KEY: 'key'>, metadata=Metadata(description=None, line_number=None, concept_source=<ConceptSource.MANUAL: 'manual'>), lineage=None, namespace='passenger', keys=None, grain=Grain(nested=False, components=[]))]))])), modifiers=[]), ColumnAssignment(alias='age', concept=Concept(name='age', dat

In [7]:
results = test.execute_text('select passenger.survived, count(passenger.id)->count_survived;')

for row in results[0].fetchall():
    print(row)

(0, 549)
(1, 342)


In [8]:
families = test.execute_text("select passenger.name limit 100;")

# query = test.generate_sql("select split(passenger.name, ',')[0]-> passenger.family;")

# print(query[0])
for row in families[0].fetchall():
    print(row)

('Braund, Mr. Owen Harris',)
('Cumings, Mrs. John Bradley (Florence Briggs Thayer)',)
('Heikkinen, Miss. Laina',)
('Futrelle, Mrs. Jacques Heath (Lily May Peel)',)
('Allen, Mr. William Henry',)
('Moran, Mr. James',)
('McCarthy, Mr. Timothy J',)
('Palsson, Master. Gosta Leonard',)
('Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',)
('Nasser, Mrs. Nicholas (Adele Achem)',)
('Sandstrom, Miss. Marguerite Rut',)
('Bonnell, Miss. Elizabeth',)
('Saundercock, Mr. William Henry',)
('Andersson, Mr. Anders Johan',)
('Vestrom, Miss. Hulda Amanda Adolfina',)
('Hewlett, Mrs. (Mary D Kingcome) ',)
('Rice, Master. Eugene',)
('Williams, Mr. Charles Eugene',)
('Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)',)
('Masselmani, Mrs. Fatima',)
('Fynney, Mr. Joseph J',)
('Beesley, Mr. Lawrence',)
('McGowan, Miss. Anna "Annie"',)
('Sloper, Mr. William Thompson',)
('Palsson, Miss. Torborg Danira',)
('Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)',)
('Emir, Mr. Farred Chehab',)
('Fortune,

In [9]:
results = test.execute_raw_sql('''
SELECT
    local_raw_data."name" as "passenger_name",
     STRING_SPLIT( local_raw_data."name" , ',' )[1] as split_name
FROM
    raw_titanic as local_raw_data

''')
for r in results.fetchall():
    print(r)

('Braund, Mr. Owen Harris', 'Braund')
('Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Cumings')
('Heikkinen, Miss. Laina', 'Heikkinen')
('Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'Futrelle')
('Allen, Mr. William Henry', 'Allen')
('Moran, Mr. James', 'Moran')
('McCarthy, Mr. Timothy J', 'McCarthy')
('Palsson, Master. Gosta Leonard', 'Palsson')
('Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 'Johnson')
('Nasser, Mrs. Nicholas (Adele Achem)', 'Nasser')
('Sandstrom, Miss. Marguerite Rut', 'Sandstrom')
('Bonnell, Miss. Elizabeth', 'Bonnell')
('Saundercock, Mr. William Henry', 'Saundercock')
('Andersson, Mr. Anders Johan', 'Andersson')
('Vestrom, Miss. Hulda Amanda Adolfina', 'Vestrom')
('Hewlett, Mrs. (Mary D Kingcome) ', 'Hewlett')
('Rice, Master. Eugene', 'Rice')
('Williams, Mr. Charles Eugene', 'Williams')
('Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)', 'Vander Planke')
('Masselmani, Mrs. Fatima', 'Masselmani')
('Fynney, Mr. Joseph J', 'Fynney')
('Beesley

In [10]:

test.hooks= [DebuggingHook()]
x= (test.generate_sql("""select split(passenger.name, ',')[1] -> passenger.family, 
                                   passenger.id.count,
                                   sum(passenger.fare) -> total_family_fare
                                    order by passenger.id.count desc;""")[0])
print(x)
families_split = test.execute_text("""select split(passenger.name, ',')[1] -> passenger.family, 
                                   passenger.id.count,
                                   sum(passenger.fare) -> total_family_fare
                                    order by passenger.id.count desc;""")

for row in families_split[0].fetchall():
    print(row)

[QUERY BUILD] getting source datasource for query with output ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.family<passenger.family>']


grain: Grain<passenger.family>


	[CONCEPT DETAIL - SELECT NODE] found direct select from location='raw_titanic'
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.id', 'passenger.family'] from [SelectNode<passenger.id>], have ['passenger.id']
		[CONCEPT DETAIL] Beginning sourcing loop for ['local._anon_function_input_7255585570204832<>', 'passenger.id<passenger.id>']
			[CONCEPT DETAIL] found direct select node with all 2 concepts, returning static selection
		[CONCEPT DETAIL] finished a loop iteration looking for ['local._anon_function_input_7255585570204832', 'passenger.id'] from [SelectNode<local._anon_function_input_7255585570204832,passenger.id>], have ['local._anon_function_input_7255585570204832', 'passenger.id']
		[CONCEPT DETAIL] have all concepts, have ['local._anon_function_input_7255585570204832', 'passenger.id'] from [SelectNode<local._anon_function_input_7255585570204832,passenger.id>] checking for single connected graph
		[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
		[CONCEPT DE

	[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
	[CONCEPT DETAIL] One fully connected subgraph returned, sourcing ['passenger.id', 'passenger.family'] successful.
	[CONCEPT DETAIL - MERGE NODE] Merge node not required as one parent node has all required output properties
[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.family', 'passenger.id.count', 'local.total_family_fare'] from [GroupNode<passenger.family,passenger.id.count>], have ['passenger.id.count', 'passenger.family']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.fare<>', 'passenger.family<passenger.family>']
	[CONCEPT DETAIL - SELECT NODE] found direct select from location='raw_titanic'
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.fare', 'passenger.family'] from [SelectNode<passenger.fare>], have ['passenger.fare']
		[CONCEPT DETAIL] Beginning sourcing loop for ['local._anon_function_input_7255585570204832<>', 'passenger.fare<>']
			[CONCEPT DETAIL] found direct select 

GroupNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
  MergeNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
    GroupNode<passenger.family,passenger.id.count>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
        SelectNode<passenger.id>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.id>->Grain<passenger.id>
          MergeNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
            SelectNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
              StaticSelectNode<passenger.id,passenger.name>->Grain<passenger.id>
    GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.fa

...

-----
   cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_2503791611268407 -> False -> ['passenger.family', 'passenger.id', 'passenger.fare']
-----
     cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 -> False -> ['local._anon_function_input_7255585570204832', 'passenger.id', 'passenger.fare']
-----
       cte_raw_data_at_passenger_id_8958832269400449 -> False -> ['passenger.name', 'passenger.id', 'passenger.fare']
-----


[RENDERING] [passenger.name] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_at_passenger_family_1099462592324186, have cte keys ['passenger.family', 'passenger.id', 'passenger.fare'] and datasource keys ['passenger.family', 'passenger.id', 'passenger.fare']
[RENDERING] [local._anon_function_input_7255585570204832] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_at_passenger_family_1099462592324186, have cte keys ['passenger.family', 'passenger.id', 'passenger.fare'] and datasource keys ['passenger.family', 'passenger.id', 'passenger.fare']
[RENDERING] [passenger.family] Complex reference falling back to source address
[RENDERING] [local.total_family_fare] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_at_passenger_family_1099462592324186 alias=True
[RENDERING] [local.total_family_fare] rendering lineage concept
[RENDERING] [passenger.fare] Attempting rendering on cte


WITH 
rhash_7192012207602026 as (
SELECT
    local_raw_data."name" as "passenger_name",
    local_raw_data."passengerid" as "passenger_id",
    local_raw_data."fare" as "passenger_fare"
FROM
    raw_titanic as local_raw_data
),
rhash_3487714809975377 as (
SELECT
    STRING_SPLIT( rhash_7192012207602026."passenger_name" , ',' ) as "_anon_function_input_7255585570204832",
    rhash_7192012207602026."passenger_id" as "passenger_id",
    rhash_7192012207602026."passenger_fare" as "passenger_fare"
FROM
    rhash_7192012207602026 as rhash_7192012207602026
),
rhash_2753986208953929 as (
SELECT
    rhash_3487714809975377."_anon_function_input_7255585570204832"[1] as "passenger_family",
    rhash_3487714809975377."passenger_id" as "passenger_id",
    rhash_3487714809975377."passenger_fare" as "passenger_fare"
FROM
    rhash_3487714809975377 as rhash_3487714809975377
),
rhash_3595792901960439 as (
SELECT
    count(rhash_2753986208953929."passenger_id") as "passenger_id_count",
    rhash_2753986

	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.id', 'passenger.family'] from [SelectNode<passenger.id>], have ['passenger.id']
		[CONCEPT DETAIL] Beginning sourcing loop for ['local._anon_function_input_7255585570204832<>', 'passenger.id<passenger.id>']
			[CONCEPT DETAIL] found direct select node with all 2 concepts, returning static selection
		[CONCEPT DETAIL] finished a loop iteration looking for ['local._anon_function_input_7255585570204832', 'passenger.id'] from [SelectNode<local._anon_function_input_7255585570204832,passenger.id>], have ['local._anon_function_input_7255585570204832', 'passenger.id']
		[CONCEPT DETAIL] have all concepts, have ['local._anon_function_input_7255585570204832', 'passenger.id'] from [SelectNode<local._anon_function_input_7255585570204832,passenger.id>] checking for single connected graph
		[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
		[CONCEPT DETAIL] One fully connected subgraph returned, sourcing ['local._anon_function_inp

GroupNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
  MergeNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
    GroupNode<passenger.family,passenger.id.count>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
        SelectNode<passenger.id>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.id>->Grain<passenger.id>
          MergeNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
            SelectNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
              StaticSelectNode<passenger.id,passenger.name>->Grain<passenger.id>
    GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.fa

In [11]:
ctas = test.execute_text("""
PERSIST family_info INTO family_info FROM                             
SELECT
    split(passenger.name, ',')[1] -> passenger.family, 
    passenger.id.count,
    sum(passenger.fare) -> total_family_fare
;
                         
                                   """)

for row in ctas[0].fetchall():
    print(row)

Persist statement persisting to location='family_info'

[QUERY BUILD] getting source datasource for query with output ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.family<passenger.family>', 'passenger.id.count<>', 'local.total_family_fare<passenger.family>']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.family<passenger.family>']
	[CONCEPT DETAIL - SELECT NODE] found direct select from location='raw_titanic'
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.id', 'passenger.family'] from [SelectNode<passenger.id>], have ['passenger.id']
		[CONCEPT DETAIL] Beginning sourcing loop for ['local._anon_function_input_7255585570204832<>', 'passenger.id<passenger.id>']
			[CONCEPT DETAIL] found direct select node with all 2 concepts, returning static selection
		[CONCEPT DETAIL] finished a loop iteration looking for ['local._anon_function_input_7255585570204832', 'passenge


grain: Grain<passenger.family>
GroupNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
  MergeNode<local.total_family_fare,passenger.family,passenger.id.count>->Grain<passenger.family>
    GroupNode<passenger.family,passenger.id.count>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
        SelectNode<passenger.id>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.id>->Grain<passenger.id>
          MergeNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
            SelectNode<local._anon_function_input_7255585570204832,passenger.id>->Grain<passenger.id>
              StaticSelectNode<passenger.id,passenger.name>->Grain<passenger.id>
    GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.fare>->Grain<passenger.id>
        SelectNode<passenger.fare>->Grain<passenger.id>
        SelectNo

GROUP BY 
    cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_2503791611268407.`passenger_family`
[RENDERING] [passenger.family] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_2503791611268407 alias=True
[RENDERING] [passenger.family] rendering lineage concept
[RENDERING] [local._anon_function_input_7255585570204832] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_2503791611268407 alias=False
[RENDERING] [passenger.name] not found in source map for cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_2503791611268407, have cte keys ['local._anon_function_input_7255585570204832', 'passenger.id', 'passenger.fare'] and datasource keys ['local._anon_function_input_7255585570204832', 'passenger.id', 'passenger.fare']
[RENDERING] [local._anon_function_input_7255585570204832] Complex reference falling back to source address
[RENDERING] [passenger.family] ignoring aggregate, already at target grain


-----
   cte_raw_data_at_passenger_id_at_passenger_id_at_passenger_id_2503791611268407 -> False -> ['passenger.family', 'passenger.id', 'passenger.fare']


  FROM
      cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 as cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508
  
[RENDERING] [local._anon_function_input_7255585570204832] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 alias=True
[RENDERING] [local._anon_function_input_7255585570204832] rendering lineage concept
[RENDERING] [passenger.name] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 alias=False
[RENDERING] [passenger.name] Basic reference, using source address for passenger.name
[RENDERING] [local._anon_function_input_7255585570204832] ignoring aggregate, already at target grain
[RENDERING] [passenger.id] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 alias=True
[RENDERING] [passenger.id] Basic reference, using source address for passenger.id
[RENDERING] [passenger.fare] Attempting rendering on cte_raw_data_at_passenger_id_at_passenger_id

-----
     cte_raw_data_at_passenger_id_at_passenger_id_4977222374304508 -> False -> ['local._anon_function_input_7255585570204832', 'passenger.id', 'passenger.fare']
-----
       cte_raw_data_at_passenger_id_8958832269400449 -> False -> ['passenger.name', 'passenger.id', 'passenger.fare']
-----
(667,)


In [12]:
for name, ds in test.environment.datasources.items():
    for x in ds.concepts:
        print(x)

passenger.id<passenger.id>
passenger.age<>
passenger.survived<>
passenger.passenger_class<>
passenger.name<>
passenger.fare<>
passenger.family<passenger.family>
passenger.id.count<passenger.family>
local.total_family_fare<passenger.family>


In [13]:
from preql.core.query_processor import source_query_concepts


# logger.addHandler(StreamHandler())
source = source_query_concepts([test.environment.concepts['passenger.family'].with_grain(test.environment.concepts['passenger.family'])],
                       test.environment)

DebuggingHook().process_root_strategy_node(source)

[CONCEPT DETAIL] found direct select node with all 1 concepts, returning static selection
[CONCEPT DETAIL - GROUP NODE] Output of group by node equals input of group by node ['passenger.family']
[CONCEPT DETAIL - GROUP NODE] No group by required, returning parent node


GroupNode<passenger.family>->Grain<passenger.family>
  StaticSelectNode<passenger.family>->Grain<passenger.family>


In [14]:
ctas = test.generate_sql(
    'select passenger.family, total_family_fare;'
)

print(ctas[0])

grain: Grain<passenger.family>

[QUERY BUILD] getting source datasource for query with output ['passenger.family<passenger.family>', 'local.total_family_fare<passenger.family>']
[CONCEPT DETAIL] found direct select node with all 2 concepts, returning static selection
[CONCEPT DETAIL - GROUP NODE] Output of group by node equals input of group by node ['passenger.family', 'local.total_family_fare']


[CONCEPT DETAIL - GROUP NODE] No group by required, returning parent node
[RENDERING] [passenger.family] Attempting rendering on cte_family_info_at_passenger_family_3507459002183680 alias=True
[RENDERING] [passenger.name] not found in source map for cte_family_info_at_passenger_family_3507459002183680, have cte keys ['passenger.family', 'local.total_family_fare'] and datasource keys ['passenger.family', 'local.total_family_fare']
[RENDERING] [local._anon_function_input_7255585570204832] not found in source map for cte_family_info_at_passenger_family_3507459002183680, have cte keys ['passenger.family', 'local.total_family_fare'] and datasource keys ['passenger.family', 'local.total_family_fare']
[RENDERING] [passenger.family] Complex reference falling back to source address
[RENDERING] [local.total_family_fare] Attempting rendering on cte_family_info_at_passenger_family_3507459002183680 alias=True
[RENDERING] [passenger.fare] not found in source map for cte_family_info_at_passenger_fami

GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
  StaticSelectNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
QueryDatasource<>->['passenger.family', 'local.total_family_fare']
  Datasource<>->['passenger.family', 'passenger.id.count', 'local.total_family_fare']
 cte_family_info_at_passenger_family_3507459002183680 -> False -> ['passenger.family', 'local.total_family_fare']
-----

WITH 
rhash_7363337764122963 as (
SELECT
    local_family_info."passenger_family" as "passenger_family",
    local_family_info."total_family_fare" as "total_family_fare"
FROM
    family_info as local_family_info
)
SELECT
    rhash_7363337764122963."passenger_family",
    rhash_7363337764122963."total_family_fare"
FROM
    rhash_7363337764122963



In [15]:
results = test.execute_text('''
 
                           key passenger.family.size <- count(passenger.id) by passenger.family;
                        select 
                            passenger.id,
                            passenger.name,
                            passenger.family, 
                            passenger.family.size,
                            passenger.fare / total_family_fare -> passenger_family_fare_percent
                            ;
    
                            ''')


for row in results[0].fetchall():
    print(row)

grain: Grain<passenger.id,passenger.family,passenger.name>


[QUERY BUILD] getting source datasource for query with output ['passenger.id<passenger.id>', 'passenger.name<>', 'passenger.family<passenger.family>', 'passenger.family.size<passenger.family>', 'local.passenger_family_fare_percent<passenger.id,passenger.family,passenger.name>']
[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.name<>', 'passenger.family<passenger.family>', 'passenger.family.size<passenger.family>', 'local.passenger_family_fare_percent<passenger.id,passenger.family,passenger.name>']
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.family<passenger.family>']
	[CONCEPT DETAIL - SELECT NODE] found direct select from location='raw_titanic'
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.id', 'passenger.family'] from [SelectNode<passenger.family,passenger.id>], have ['passenger.id', 'passenger.family']
	[CONCEPT DETAIL] have all concepts, have ['passenger.id', 'passenger.family'] 

	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.fare', 'local.total_family_fare', 'passenger.id', 'passenger.family', 'passenger.name'] from [GroupNode<local.total_family_fare,passenger.family>], have ['local.total_family_fare', 'passenger.family']
	[CONCEPT DETAIL - SELECT NODE] found direct select from location='raw_titanic'
	[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.fare', 'local.total_family_fare', 'passenger.id', 'passenger.family', 'passenger.name'] from [GroupNode<local.total_family_fare,passenger.family>, SelectNode<passenger.family,passenger.fare,passenger.id,passenger.name>], have ['local.total_family_fare', 'passenger.family', 'local.total_family_fare', 'passenger.family', 'passenger.fare', 'passenger.id', 'passenger.family', 'passenger.name']
	[CONCEPT DETAIL] have all concepts, have ['passenger.fare', 'local.total_family_fare', 'passenger.id', 'passenger.family', 'passenger.name'] from [GroupNode<local.total_family_fare,passenge

GroupNode<local.passenger_family_fare_percent,passenger.family,passenger.family.size,passenger.id,passenger.name>->Grain<passenger.id,passenger.family,passenger.name>
  MergeNode<local.passenger_family_fare_percent,passenger.family,passenger.family.size,passenger.id,passenger.name>->Grain<passenger.family,passenger.id>
    GroupNode<passenger.family,passenger.family.size>->Grain<passenger.family>
      MergeNode<passenger.family,passenger.id>->Grain<passenger.id>
        SelectNode<passenger.family,passenger.id>->Grain<passenger.id>
    SelectNode<passenger.family,passenger.id,passenger.name>->Grain<passenger.id>
    SelectNode<local.passenger_family_fare_percent,passenger.family,passenger.id,passenger.name>->Grain<passenger.family,passenger.id>
      MergeNode<local.total_family_fare,passenger.family,passenger.fare,passenger.id,passenger.name>->Grain<passenger.family,passenger.id>
        GroupNode<local.total_family_fare,passenger.family>->Grain<passenger.family>
          MergeNode<

            raw_titanic as local_raw_data
        
[RENDERING] [passenger.fare] Attempting rendering on cte_raw_data_at_passenger_id_8958832269400449 alias=True
[RENDERING] [passenger.fare] Basic reference, using source address for passenger.fare
[RENDERING] [passenger.id] Attempting rendering on cte_raw_data_at_passenger_id_8958832269400449 alias=True
[RENDERING] [passenger.id] Basic reference, using source address for passenger.id
[RENDERING] [passenger.family] Attempting rendering on cte_raw_data_at_passenger_id_8958832269400449 alias=True
[RENDERING] [passenger.family] rendering lineage concept
[RENDERING] [local._anon_function_input_7255585570204832] Attempting rendering on cte_raw_data_at_passenger_id_8958832269400449 alias=False
[RENDERING] [local._anon_function_input_7255585570204832] rendering lineage concept
[RENDERING] [passenger.name] Attempting rendering on cte_raw_data_at_passenger_id_8958832269400449 alias=False
[RENDERING] [passenger.name] Basic reference, using source 

-----
(478, 'Braund, Mr. Lewis Richard', 'Braund', 2, 0.49285804222219115)
(2, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'Cumings', 1, 1.0)
(3, 'Heikkinen, Miss. Laina', 'Heikkinen', 1, 1.0)
(731, 'Allen, Miss. Elisabeth Walton', 'Allen', 2, 0.9633069340778303)
(568, 'Palsson, Mrs. Nils (Alma Cornelia Berglund)', 'Palsson', 4, 0.25)
(870, 'Johnson, Master. Harold Theodor', 'Johnson', 6, 0.2703904563216911)
(123, 'Nasser, Mr. Nicholas', 'Nasser', 2, 0.5)
(886, 'Rice, Mrs. William (Margaret Norton)', 'Rice', 5, 0.2)
(334, 'Vander Planke, Mr. Leo Edmondus', 'Vander Planke', 3, 0.3333333333333333)
(20, 'Masselmani, Mrs. Fatima', 'Masselmani', 1, 1.0)
(21, 'Fynney, Mr. Joseph J', 'Fynney', 1, 1.0)
(22, 'Beesley, Mr. Lawrence', 'Beesley', 1, 1.0)
(23, 'McGowan, Miss. Anna "Annie"', 'McGowan', 1, 1.0)
(262, 'Asplund, Master. Edvin Rojj Felix', 'Asplund', 4, 0.25)
(27, 'Emir, Mr. Farred Chehab', 'Emir', 1, 1.0)
(439, 'Fortune, Mr. Mark', 'Fortune', 4, 0.25)
(32, 'Spencer, Mrs. Wil